In [1]:
import numpy as np
import itertools
from itertools import cycle, islice
import operator

Function to put brackets around left and right parts of the expression (only if the length of the part is more than 1).

In [161]:
def put_brackets(left_part, right_part, operator):
    if len(left_part)> 1 and len(right_part) > 1:
        return "("+left_part+")"+operator+"("+right_part+")"
    if len(left_part)> 1 and len(right_part) == 1:
        return "("+left_part+")"+operator+right_part
    if len(left_part)== 1 and len(right_part) > 1:
        return left_part+operator+"("+right_part+")"
    if len(left_part)== 1 and len(right_part) == 1:
        return left_part+operator+right_part

Minimizing recursive function

In [192]:
def minimize(expr):
    # base case
    if len(expr)==1:
        if expr.isdigit():
            return int(expr), expr
        else:
            return "Invalid symbol"
    #variable to put final result. It is infinity in the beginning to be sure that at first comparison
    #between res and our expression result, res would be overwritten.
    res = np.inf
    #variable to put our final expression as string with brackets
    final_expr = ""
    # iterating over the expression, looking for operators
    for i in range(len(expr) - 1):
        if expr[i] == '+':
            #expression before operator
            left_expr = expr[:i]
            #expression after operator
            right_expr = expr[i + 1 :len(expr)]
            #computing their values by recursively calling the minimize function
            left_value, left_part = minimize(left_expr)
            right_value, right_part = minimize(right_expr)
            # if the sum of values is less than our minimal result before, this solution becomes new minimal result
            if left_value + right_value < res:
                res = left_value + right_value
                final_expr = put_brackets(left_part, right_part, expr[i])
        if expr[i] == '*':
            left_expr = expr[:i]
            right_expr = expr[i + 1 :len(expr)]
            left_value, left_part = minimize(left_expr)
            right_value, right_part = minimize(right_expr)
            if left_value * right_value < res:
                res = left_value * right_value
                final_expr = put_brackets(left_part, right_part, expr[i])
        if expr[i] == '-':
            left_expr = expr[:i]
            right_expr = expr[i + 1 :len(expr)]
            left_value, left_part = minimize(left_expr)
            right_value, right_part = minimize(right_expr)
            if left_value - right_value < res:
                res = left_value - right_value
                final_expr = put_brackets(left_part, right_part, expr[i])
        if expr[i] == '/':
            left_expr = expr[:i]
            right_expr = expr[i + 1 :len(expr)]
            left_value, left_part = minimize(left_expr)
            right_value, right_part = minimize(right_expr)
            #preventing division by zero
            if right_value != 0:
                if left_value / right_value < res:
                    res = left_value / right_value
                    final_expr = put_brackets(left_part, right_part, expr[i])
    return res, final_expr

In [181]:
minimize("1*3-2*5/1")

(-7.0, '1*(3-(2*(5/1)))')

Maximizing recursive function that works exactly as the minimizing one only the other way around.

In [190]:
def maximize (expr):
    if len(expr)==1:
        if expr.isdigit():
            return int(expr), expr
        else:
            return "Invalid symbol"
    res = np.NINF
    final_expr = ""
    for i in range(len(expr) - 1):
        left_expr, right_expr = "", ""
        if expr[i] == '+':
            left_expr = expr[:i]
            right_expr = expr[i + 1 :len(expr)]
            left_value, left_part = maximize(left_expr)
            right_value, right_part = maximize(right_expr)
            if left_value + right_value > res:
                res = left_value + right_value
                final_expr = put_brackets(left_part, right_part, expr[i])
        if expr[i] == '*':
            left_expr = expr[:i]
            right_expr = expr[i + 1 :len(expr)]
            left_value, left_part = maximize(left_expr)
            right_value, right_part = maximize(right_expr)
            if left_value * right_value > res:
                res = left_value * right_value
                final_expr = put_brackets(left_part, right_part, expr[i])
        if expr[i] == '-':
            left_expr = expr[:i]
            right_expr = expr[i + 1 :len(expr)]
            left_value, left_part = maximize(left_expr)
            right_value, right_part = maximize(right_expr)
            if left_value - right_value > res:
                res = left_value - right_value
                final_expr = put_brackets(left_part, right_part, expr[i])
        if expr[i] == '/':
            left_expr = expr[:i]
            right_expr = expr[i + 1 :len(expr)]
            left_value, left_part = maximize(left_expr)
            right_value, right_part = maximize(right_expr)
            if right_value != 0:
                if left_value / right_value > res:
                    res = left_value / right_value
                    final_expr = put_brackets(left_part, right_part, expr[i])
    return res, final_expr

In [183]:
maximize('1*3-2*5/1')

(5.0, '1*((3-2)*(5/1))')

Function that takes a list of numbers and list of operators and returns all possible combinations of operators that can be between these numbers with possible repetitions.

In [184]:
def put_operators(numbers, operators):
    oper_nr = len(numbers)-1
    opers = list(itertools.product(operators, repeat=oper_nr))
    return opers

In [185]:
put_operators([1, 2, 3, 4], ['*', '+'])

[('*', '*', '*'),
 ('*', '*', '+'),
 ('*', '+', '*'),
 ('*', '+', '+'),
 ('+', '*', '*'),
 ('+', '*', '+'),
 ('+', '+', '*'),
 ('+', '+', '+')]

From https://docs.python.org/2/library/itertools.html

In [10]:
def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    # Recipe credited to George Sakkis
    num_active = len(iterables)
    nexts = cycle(iter(it).__next__ for it in iterables)
    while num_active:
        try:
            for next in nexts:
                yield next()
        except StopIteration:
            # Remove the iterator we just exhausted from the cycle.
            num_active -= 1
            nexts = cycle(islice(nexts, num_active))

The final function that takes numbers, possible operators and maximize/minimize function as arguments and returns the maximal/minimal result and how it was achieved.

In [193]:
def put_operators_final(numbers, operators, func):
    results={}
    #possible operators' combinations with repetitions
    opers = put_operators(numbers, operators)
    for oper in opers:
        #creating a possible expression preserving numbers' order
        str_expr = ''.join(str(e) for e in roundrobin(numbers, oper))
        #saving expression as dictionary key with its result as value
        results[str_expr] = func(str_expr)
    if func == maximize:
        #looking for max result
        key = max(results, key=lambda k: results[k][0])
    if func == minimize:
        #looking for min result
        key = min(results, key=lambda k: results[k][0])
    return results[key]

In [194]:
put_operators_final([1, 2, 3, 4, 5], ['*', '+', '-', '/'], minimize)

(-119, '1-(2*(3*(4*5)))')

In [195]:
put_operators_final([2, 1, 7, 1, 4, 3], ['*', '+'], maximize)

(315, '(2+1)*(7*((1+4)*3))')